In [1]:
import pandas as pd
import numpy as np


In [5]:
d = pd.read_excel("simplified.xlsx")

In [14]:
d
print(" ")
d["Unnamed: 0"].str.replace("", " ").split(",")

AttributeError: 'Series' object has no attribute 'split'